<a href="https://colab.research.google.com/github/mchowdh200/exome-copy/blob/master/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

#!pip install --upgrade pandas
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load/Process Data

In [0]:
deletions = pd.read_pickle('/content/gdrive/My Drive/Colab Notebooks/deletions.pkl')

In [0]:
duplications = pd.read_pickle('/content/gdrive/My Drive/Colab Notebooks/duplications.pkl') 

In [0]:
non_sv = pd.read_pickle('/content/gdrive/My Drive/Colab Notebooks/non_sv.pkl') 

In [6]:
print(non_sv.data.values.shape)
print(deletions.data.values.shape)
print(duplications.data.values.shape)

(105339,)
(48939,)
(19485,)


In [0]:
data = np.concatenate((
    non_sv.data.values,
    deletions.data.values,
    duplications.data.values
))

labels = np.concatenate((
    np.zeros((len(non_sv),)),
    np.full((len(deletions,)), fill_value=1),
    np.full((len(duplications,)), fill_value=2)
))

In [0]:
labels = to_categorical(labels)

In [0]:
#data[0].shape
#print(pad_sequences(data, maxlen=225).shape)
data_padded = [pad_sequences(d, maxlen=300) for d in data]

In [0]:
data_padded = np.array(data_padded)

In [11]:
data_padded.shape

(173763, 8, 300)

## Define Model

In [0]:
model = Sequential([
    Conv1D(input_shape=data_padded.shape[1:],
           filters=512,
           kernel_size=6,
           strides=1,
           dilation_rate=1,
           data_format='channels_first',
           kernel_initializer='glorot_uniform',
           activation=LeakyReLU()),
    Dropout(0.25),
    MaxPool1D(pool_size=3),
    Conv1D(filters=128,
           kernel_size=6,
           strides=1,
           dilation_rate=1,
           data_format='channels_first',
           kernel_initializer='glorot_uniform',
           activation=LeakyReLU()),
    Dropout(0.25),
    MaxPool1D(pool_size=3),
    Flatten(),
    Dense(3, activation='softmax')
])

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
X_train, X_val, y_train, y_val = train_test_split(data_padded, labels, stratify=labels, test_size=0.1)
callbacks = [EarlyStopping(patience=4),
             ReduceLROnPlateau(patience=2)]
model.fit(X_train, y_train,
          epochs=20,
          batch_size=512,
          verbose=1,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

Train on 156386 samples, validate on 17377 samples
Epoch 1/20
156386/156386 [==============================] - 34s 218us/sample - loss: 3.0050 - acc: 0.6664 - val_loss: 2.6359 - val_acc: 0.3688
Epoch 2/20
156386/156386 [==============================] - 33s 213us/sample - loss: 0.6708 - acc: 0.7742 - val_loss: 0.6245 - val_acc: 0.7965
Epoch 3/20
156386/156386 [==============================] - 34s 214us/sample - loss: 0.5961 - acc: 0.7999 - val_loss: 0.5789 - val_acc: 0.8145
Epoch 4/20
156386/156386 [==============================] - 34s 216us/sample - loss: 0.5656 - acc: 0.8105 - val_loss: 0.5772 - val_acc: 0.8294
Epoch 5/20
156386/156386 [==============================] - 34s 218us/sample - loss: 0.5466 - acc: 0.8178 - val_loss: 0.5836 - val_acc: 0.8184
Epoch 6/20
156386/156386 [==============================] - 34s 219us/sample - loss: 0.5377 - acc: 0.8237 - val_loss: 0.6167 - val_acc: 0.8356
Epoch 7/20
156386/156386 [==============================] - 34s 219us/sample - loss: 0.4804